In [80]:
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import psutil
import plotly.graph_objects as go
import chart_studio
import chart_studio.plotly as py

import plotly.io as pio

chart_studio.tools.set_config_file(world_readable=True,
                             sharing='public')


In [81]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [82]:
stonks = pd.read_csv("stonks.csv")

In [83]:
stonks = stonks.iloc[::-1]
stonks.head()

,timestamp,open,high,low,close,volume
5032,2000-03-29,151.5625,152.4843,149.6562,151.2187,6747500
5031,2000-03-30,150.1562,151.9218,147.1250,148.6875,9491900
5030,2000-03-31,149.6250,152.3125,148.4375,150.3750,9249100
5029,2000-04-03,150.1250,151.2500,148.6875,151.2500,8508200
5028,2000-04-04,151.7500,153.0000,141.3906,150.1250,19585500


In [108]:
split_percent = 0.7
val_percent = 0.1

close_data = stonks[["open", "high", "low", "close", "close"]].values.reshape((-1,5))
scaler = MinMaxScaler()
scaler.fit(close_data)

close_data = scaler.transform(close_data)
split = int(split_percent*len(close_data))
val = int((val_percent + split_percent) * len(close_data))
close_train = close_data[:split]
close_val = close_data[split:val]
close_test = close_data[val:]

date_train = stonks.timestamp[:split]
date_val = stonks.timestamp[split:val]
date_test = stonks.timestamp[val:]

close_train[0:, 4:]

array([[0.30754801],
       [0.29818118],
       [0.30442586],
       ...,
       [0.43437072],
       [0.43999556],
       [0.44458424]])

In [114]:
look_back = 20
num_epochs = 92

train_generator = keras.preprocessing.sequence.TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)   
val_generator = keras.preprocessing.sequence.TimeseriesGenerator(close_val, close_val, length=look_back, batch_size=20)     
test_generator = keras.preprocessing.sequence.TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

model = keras.Sequential([
    keras.layers.LSTM(256,
                      activation="relu",
                      #recurrent_activation="sigmoid",
                      use_bias=True,
                      input_shape=(look_back, 5)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(5),
])
model.compile(optimizer="adam", loss="mse", metrics=["accuracy"])


model.fit(train_generator, epochs=num_epochs, verbose=1, validation_data=val_generator)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 176 steps, validate for 25 steps
Epoch 1/92
176/176 [==============================] - 2s 14ms/step - loss: 0.0026 - accuracy: 0.1970 - val_loss: 7.0307e-04 - val_accuracy: 0.0000e+00
Epoch 2/92
176/176 [==============================] - 2s 10ms/step - loss: 5.3889e-04 - accuracy: 0.2124 - val_loss: 1.8375e-04 - val_accuracy: 0.4576
Epoch 3/92
176/176 [==============================] - 2s 10ms/step - loss: 4.7037e-04 - accuracy: 0.2210 - val_loss: 2.0074e-04 - val_accuracy: 0.4576
Epoch 4/92
176/176 [==============================] - 2s 10ms/step - loss: 3.6491e-04 - accuracy: 0.2261 - val_loss: 0.0020 - val_accuracy: 0.3147
Epoch 5/92
176/176 [==============================] - 2s 10ms/step - loss: 3.2501e-04 - accuracy: 0.2318 - val_loss: 8.4932e-04 - val_accuracy: 0.0414
Epoch 6/92
176/176 [==============================] - 2s 10ms/step - loss: 2.7668e-04 - accuracy: 0.2192 - val_loss: 1.7163e-04 - val_accuracy: 0.0352
Epoch

In [117]:
prediction = model.predict(test_generator)

def denorm(d): 
    return scaler.inverse_transform(d)[0:, 4:].reshape((-1))
close_train_1 = denorm(close_train)
close_val_1 = denorm(close_val)
close_test_1 = denorm(close_test)
prediction_1 = denorm(prediction)

diff = close_test_1[look_back:] - prediction_1
avg_diff = np.mean(diff)
print(avg_diff)

def predict(num_prediction, before_lookback, model):
    if (before_lookback > 0):
        prediction_list = close_data[-look_back - before_lookback:-before_lookback]
    else:
        prediction_list = close_data[-look_back:]
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
#         print("Pred list", prediction_list)
        x = x.reshape((1, look_back, 5))
        out = model.predict(x)
#         print("Out", out)
        prediction_list = np.append(prediction_list, out, axis=0)
    prediction_list = prediction_list[look_back:]
        
    return prediction_list
    
def predict_dates(num_prediction, before_lookback):
    last_date = stonks.timestamp.values[-1 - before_lookback]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1+before_lookback).tolist()
    return prediction_dates

num_prediction = 4
before_lookback = 0
forecast = denorm(predict(num_prediction, before_lookback, model))
forecast_dates = predict_dates(num_prediction, before_lookback)
prediction_1 = np.concatenate([[None] * look_back, prediction_1 + avg_diff])
trace1 = go.Scatter(
    x = np.concatenate([date_train, date_val]),
    y = np.concatenate([close_train_1, close_val_1]),
    mode = 'lines',
    name = 'Real Data - trained'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction_1,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test_1,
    mode='lines',
    name = 'Real Data - untrained'
)
forecast = go.Scatter(
    x = forecast_dates,
    y = forecast + avg_diff,
    mode = 'lines+markers+text',
    texttemplate = "%{y:.2f}",
    name = 'Forecast'
)

layout = go.Layout(
    title = "SPY",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3, forecast], layout=layout)
fig.show(renderer="browser")
# py.plot(fig, filename = 'basic-line', renderer="browser", auto_open=True)

1.9219494742195415


In [39]:
pio.write_html(fig, file='tracing_pred.html', auto_open=True)


In [34]:
model.save('good_model_bak_2.h5')

In [37]:
model = keras.models.load_model("good_model_bak_2.h5", custom_objects={'loss': customloss()})